<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Decision Trees
_Author: Kevin Coyle_

### LEARNING OBJECTIVES
*After this lesson, you will be able to:*
- Describe how tree based models are created for classification tasks and regression tasks 
- Gain intuition on the math behind these models
- Implement a tree model in Python
- Implement a tree model in Python using SciKit Learn

### Decision trees

Decision trees are extremely intuitive ways to classify or label objects: you simply ask a series of questions designed to zero-in on the classification. 

For example, if you wanted to build a decision tree to classify an animal you come across while on a hike, you might construct the one shown here:

<img src = "../figures/decision_tree.png">

There are several algorithms to implement decision trees mathematically. We're using the classification and regression tree (CART) algorithm.
Other algorithms that make decision trees:
- [ID3](https://medium.com/machine-learning-guy/an-introduction-to-decision-tree-learning-id3-algorithm-54c74eb2ad55)
- [C4.5](https://towardsdatascience.com/what-is-the-c4-5-algorithm-and-how-does-it-work-2b971a9e7db0)
- [C5.0](https://www.ibm.com/support/knowledgecenter/en/SS3RA7_15.0.0/com.ibm.spss.modeler.help/c50node_general.htm)

#### Does this mean that trees can perform classification _and_ regression tasks? 

Great. Let's go into the math, then the Python, then the SK Learn

### The Math

Decision trees primarily rely on two equations to split data in a _binary recursive_ fashion:
- Gini Impurity
- Information Gain

Recall our attempt to classify an animal above. Trees are trying to reduce the possibility of mislabeling an observation, given the data. We can think of it as a question about the features (scroll up to see the example with the animals... ex. "How big is the animal?"). 

"The best question is the one that reduces our uncertainty the most."

These two equations are trying to answer two questions:
1. Can we quantify how much uncertainty we have at a node (Gini Impurity)?
2. How much uncertainty did we just reduce, once we split our data (Information Gain)?

#### Gini Impurity

- Our goal is to measure how much "uncertainty" or "potential to mislabel a target, given the data"
- At the root node, our data has however many classes of labels. In our toy dataset, we have 3 labels.
- A node is completely pure at 0 (0 == no uncertainty) and there is more impurity as Gini Impurity rises

Imagine we have a bucket with childrens blocks in the bucket. 

<img src = "https://i.etsystatic.com/9927164/r/il/d9e833/1518306239/il_1588xN.1518306239_6mf0.jpg" height="400" width="400">

Now imagine we have those 3 blocks, and each block is stamped with the letter "A" on every side of the block, and those 3 blocks with the letter A are all in one bucket. Also, we have a hat that has 3 pieces of paper (our labels) in the hat, and each piece of paper has a label written on it, that says "A." 

If we pull a block out of the bucket and pull a label out of the hat, we will always obtain the correct letter. There are 3 blocks, and there are 3 corresponding labels.

Our Proportion (p) - in other words, our "ratio" - is 3/3 (or 1). We have no way to make a mistake here, so our impurity is $0$

Formally defined:

Gini Impurity = 
$\sum_{j=1}^{c} = 1 - p_i$

Let's implement this math with our example of the blocks

Great. Now we know that Gini Impurity is how we classify how "pure" or "impure" our node is, given the number of labels (also called: "classes"), and our proportion of data within those classes.

#### Would a Gini Impurity score of 0.2 be more pure or less pure than a score of 0.21? 

Try this again with our example of 3 blocks, and this time, assume that we have 1 block for 3 letters: A, B, and C (iow: 1 bucket has 3 blocks in it and each block as a different letter on it).

Now in the hat, we have 3 pieces of paper. One piece of paper says "A" on it, the second piece of paper says "B" on it, and the third piece of paper says "C" on it.

Calculate this, with the Gini Impurity formula from above, and our _new proportion_

#### Information Gain

Recall that we're trying to answer two questions to determine the "best split" in our data:
1. How much uncertainty do we have in our node at a current point in time?
2. If we make a split, how much did we just reduce our uncertainty by?

This second question is "information gain." Rolled up into this one question though is an inherent sub question:
"How much uncertainty was there to begin with?"


To solve this sub question, we turn to a new formula called "Entropy"

Paraphrashing from our compatriots in mathematical arms, the physicists, we can loosely define Entropy as _"an amount of disorder."_

Here, we can rephrase it as "entropy is the amount of messiness in our data classification." 
<img src = "https://c8.alamy.com/comp/AB6DGB/extremely-messy-room-of-a-teenage-AB6DGB.jpg" height=400 width=400>
(lots of entropy)

Our goal is to sort/tidy this data. Entropy is highly related to Gini Impurity, but they are calculated differently and have different objectives (mostly having to do with the fact that we want to find an average weight of entropy because we care more about a large set with a low set of uncertainty, than a small set with a high amount of uncertainty).

Formally defined.

$Entropy:\; H(C) = -\sum p(C)\log p(C)$



Once we know how much entropy exists in our node, we can calculate how much a split would effect our information gain. We'll keep track of this amount, and the split that gives us the most gain is our best split (or "is the best question to reduce our uncertainty").

This second half of the equation in Information Gain is could also be called [conditional entropy](https://en.wikipedia.org/wiki/Conditional_entropy) iow: given this new split, how much entropy are we expected to have?

Formally defined.

$Information\ Gain:\; I(C,Y)= H(C)-H(C|Y)$

or

Information Gain = entropy(the parent node) - the weighted average * entropy(the child node)

The weighted average can be found like:
Weighted avg] Entropy(children) = 

$(Number\ of\ examples\ in\ left\ child\ node)\ /\ (Total\ number\ of\ examples\ in\ parent\ node)\ *\ (entropy\ of\ left\ node)$ 

+

$(Number\ of\ examples\ in\ right\ child\ node)\ /\ (Total\ number\ of\ examples\ in\ parent\ node)\ *\ (entropy\ of\ right\ node)$

#### What question is information gain trying to answer?

# Decision Trees in SciKit Learn

Our goal is to always understand our models and what is happening within them, down to the code.

However, it is often prudent to stand on the shoulders of giants for our initial prototyping. Let's learn how to use SK Learn's Decision Tree classifier!

Import our data. Today, we're predicting the outcome of a March Madness game

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pwd

In [ ]:
ncaa_df = pd.read_csv('../data/ncaa.csv')

In [ ]:
ncaa_df.head().T

Check out the dtypes and summary statistics for data. Also, are there any NaNs? Handle those if so!

Let's drop the Rank column

In [ ]:
ncaa_df.drop(columns=['Rank'], inplace=True)

We want to change the Location into a number. To acheive:
- Use Pandas get dummies to create a new dataframe called "location_dummies."
- You can drop one of the locations if you wish (recall: multicollinearity)
- Join this location_dummies back onto our dataframe, then drop the original column

Create an `X`and a `y`. 

In [ ]:
feature_cols = ['Seed', 'STL', 'PPG_Diff', 'PPG']
X = ncaa_df[feature_cols]
y = ncaa_df.Result

In [ ]:
ncaa_df.Result.describe()

Train test split that data! First, import your train_test_split function in sklearn.model_selection.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Import Decision Tree Classifier from SkLearn.tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

Instantiate our model. Also, let's check out some of the hyper-parameters

In [ ]:
DT = DecisionTreeClassifier()

In [ ]:
DT

Recall that sklearn has some excellent default settings. One of these is Gini! Write out a couple of the hyper parameters in the cell below. We're going to go over them in a moment, so don't worry if you don't know what these do yet!

Fit the model.

In [ ]:
DT.fit(X_train, y_train)

Let's save these predictions to a variable called ŷ (y_hat)

In [ ]:
y_pred = DT.predict(X_test)

In [ ]:
sum(abs(y_pred - y_test))

### Some hyper parameters

Recall there were several hyperparameters available to us in sklearn. We're now going to focus on a few important ones. To set a hyper parameter in SK Learn, you need to instantiate the model with that hyper parameter.

In [ ]:
feature_cols = ['Seed', 'STL', 'PPG_Diff', 'PPG']
X = ncaa_df[feature_cols]
y = ncaa_df.Result

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#reinstantiate model
DT_2 = DecisionTreeClassifier(min_samples_split=4)

In [ ]:
#fit that model
DT_2.fit(X_train, y_train)

In [ ]:
#check results
y_pred = DT_2.predict(X_test)

In [ ]:
sum(abs(y_pred - y_test))

Decision trees can be great models, and they form the basis of more robust, powerful models. We'll go over other tree based models at a later point in class (like bagged trees and random forests).

Their real distinction though, is their interpretability. You know what decision trees look like if you craft them by hand... but what if you could do that with the model you just fit?

You can! Graphviz will allow us to do this. You'll create a new file type: a dot file. Then you convert this to an image file and you're golden!

First, install:

In [ ]:
!conda install graphviz --yes

`export_graphviz` is a function in sklearn's tree module... import it!

Then:

import graphviz, which will allow us to change the dot file into a PDF or PNG

The following code exports our tree as dot file, and then converts it back to a png!

In [ ]:
dot_data = export_graphviz(dec_tree, out_file=None, 
                     feature_names= X.columns,  
                     class_names= "result", 
                     filled=True, rounded=True,  
                     special_characters=True)  
graph = graphviz.Source(dot_data)  
#this last line creates a PDF of your diagram. You can change the format to "png" if you want an image
graph.render("ncaa", format='png') 

Cool! Now you can show your friends how you chose your NCAA bracket and have some machine learning to back your analysis. :D

## Recap

#### What does "entropy" refer to?

#### Can you name one of the two hyper parameters we discussed for decision trees (it's okay to look if you can't remember!)?

#### Can you name the other one?

#### I have 4 items: a basketball, a soccer ball, a frisbee, and a tennis ball in a bucket and 4 labels of each of those in another container. If I pull 1 item from the bucket and 1 item from the other container, what is the ratio/proportionality that I may choose the correct label?

## Advantages of trees:
- Trees are very easy to explain to people. In fact, they are even easier to explain than linear regression!
- Some people believe that decision trees more closely mirror human decision-making than do the regression and classification approaches seen in previous chapters.
- Trees can be displayed graphically, and are easily interpreted even by a non-expert (especially if they are small).

## Disadvantages of trees:
- Unfortunately, trees generally do not have the same level of predictive accuracy as some of the other regression and classification approaches seen in this book.
- Additionally, trees can be very non-robust. In other words, a small change in the data can cause a large change in the final estimated tree.

However, by aggregating many decision trees, using methods like bagging, random forests, and boosting, the predictive performance of trees can be substantially improved. We introduce these concepts in a later lesson!